In [1]:
%load_ext autoreload
%autoreload 2
import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve,f1_score
from ml_utils.metrics import aur_pr
from answer_judge import load_judge

In [2]:
data_train=ReadData('/root/autodl-tmp/hallscope/AOKVQA/train').read_all()
random.shuffle(data_train)
#data_val=ReadData('/root/autodl-fs/hallscope/ScienceQA/val').read_all()
#data_test=ReadData('/root/autodl-fs/hallscope/ScienceQA/test').read_all()

In [3]:
df_train=pd.DataFrame(data_train)
#df_val=pd.DataFrame(data_val)
#df_test=pd.DataFrame(data_test)

In [4]:
df_train.head()

,img_path,question,answer,most_likely,responses
0,[/root/autodl-tmp/coco2017/train2017/000000040...,[Question:What is she doing?\nAnswer the Quest...,"[tensor(1), tensor(2), tensor(1), tensor(2), t...",{'embedding': [[[ 7.2813034e-04 6.3419342e-04...,None
1,[/root/autodl-tmp/coco2017/train2017/000000374...,[Question:What is the women about to do?\nAnsw...,"[tensor(2), tensor(3), tensor(1), tensor(3), t...",{'embedding': [[[ 7.2813034e-04 6.3419342e-04...,None
2,[/root/autodl-tmp/coco2017/train2017/000000227...,[Question:The company advertised makes which o...,"[tensor(2), tensor(0), tensor(2), tensor(2), t...",{'embedding': [[[ 7.2813034e-04 6.3419342e-04...,None
3,[/root/autodl-tmp/coco2017/train2017/000000000...,[Question:What is the woman in yellow waiting ...,"[tensor(2), tensor(0), tensor(2), tensor(3), t...",{'embedding': [[[ 7.2813034e-04 6.3419342e-04...,None
4,[/root/autodl-tmp/coco2017/train2017/000000287...,[Question:At which building does the purple tr...,"[tensor(0), tensor(0), tensor(2), tensor(2), t...",{'embedding': [[[ 7.2813034e-04 6.3419342e-04...,None


In [5]:
i=0
df_train.iloc[i]['most_likely']['response']

['B', 'C', 'B', 'C', 'D.duck', 'B.electricity', 'D', 'D']

In [6]:
df_train.iloc[i]['img_path']

['/root/autodl-tmp/coco2017/train2017/000000040446.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000227495.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000204024.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000349647.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000428595.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000023047.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000484369.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000567220.jpg']

In [7]:
df_train.iloc[i]['question']

['Question:What is she doing?\nAnswer the Question with following choices.\nChoices: A.resting B.praying C.hiding D.eating\n',
 'Question:What is the man wearing?\nAnswer the Question with following choices.\nChoices: A.glasses B.clown nose C.headband D.gas mask\n',
 'Question:What is the person skiing on?\nAnswer the Question with following choices.\nChoices: A.ice B.water C.snow D.sand\n',
 'Question:What will the woman drink with her pizza?\nAnswer the Question with following choices.\nChoices: A.milk B.coke C.beer D.wine\n',
 'Question:What do the women here find most interesting?\nAnswer the Question with following choices.\nChoices: A.walkway B.goose C.child D.duck\n',
 'Question:What energy powers the blender?\nAnswer the Question with following choices.\nChoices: A.solar B.electricity C.battery D.manual\n',
 "Question:What is used for the bear's eye?\nAnswer the Question with following choices.\nChoices: A.lid B.rock C.coin D.button\n",
 'Question:Which towel has been used rece

In [8]:
df_train.iloc[0]['answer']

tensor([1, 2, 1, 2, 2, 1, 3, 3])

In [9]:
embeddings=df_train['most_likely'].apply(lambda x: x['embedding']).to_list()
embeddings=np.concatenate(embeddings,axis=0)
embeddings.shape

(17056, 33, 4096)

In [10]:
gt_label=df_train['answer'].to_list()
gt_label=np.concatenate(gt_label,axis=0)
gt_label.shape

(17056,)

In [11]:
gt_label[0]

np.int64(1)

In [12]:
judge=load_judge('OKVQA')

In [ ]:
responses=df_train['most_likely'].apply(lambda x: x['response']).to_list()

rsps=[]
for x in responses:
    rsps+=x
    
is_correct=[]
for i,x in enumerate(rsps):
    is_correct.append(judge.check(x,gt_label[i]))

is_correct=np.array(is_correct)
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 17056 responses. 11081 answer correctly and 5975 answer wrongly


In [ ]:
length = len(embeddings)
train_rate = 0.8
num_val=0

embedding_train = embeddings[:int(length*train_rate)-num_val]
embedding_val = embeddings[int(length*train_rate)-num_val:int(length*train_rate)]
embedding_test = embeddings[int(length*train_rate):]


y_train = is_correct[:int(length*train_rate)-num_val]
y_val = is_correct[int(length*train_rate)-num_val:int(length*train_rate)]
y_test = is_correct[int(length*train_rate):]


'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
print(f'trainset length:{len(y_train)}')
print(f'validset length:{len(y_val)}')
print(f'testset length:{len(y_test)}')
print(
    f'trainset postive: {sum(y_train==1)} negative: {sum(y_train==0)}')
print(
    f'valset postive: {sum(y_val==1)} negative: {sum(y_val==0)}')
print(
    f'testset postive: {sum(y_test==1)} negative: {sum(y_test==0)}')

trainset length:13644
validset length:0
testset length:3412
trainset postive: 8845 negative: 4799
valset postive: 0 negative: 0
testset postive: 2236 negative: 1176


In [15]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCADiscriminator
from ml_utils.metrics import auroc,get_best_split_from_scores

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [16]:
best_layer=10

In [17]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=y_test

In [18]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [19]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores = torch.sigmoid(output).cpu().data.numpy()
preds=scores>0.5
print(f'test set acc: {accuracy_score(y_test,preds)} AUROC:{roc_auc_score(y_test,scores)} AUC-PR:{aur_pr(scores,y_test)} f1:{f1_score(y_test,preds)}')

test set acc: 0.6834701055099648 AUROC:0.6873982710257627 AUC-PR:0.7948696401040842 f1:0.7805770012190166
